In [36]:
import json
import numpy as np
import argparse
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import f1_score

In [37]:
method = 'Demo'

In [38]:
with open(f'eval_{method}_results.json', 'r') as f:
        eval_results = json.load(f)
all_instances = []
for key in eval_results:
    all_instances.extend(eval_results[key])
all_instances = list(set(all_instances))
# with open(f'predicted_{method}_results.json', 'r') as f:
#     static_predicted_results = json.load(f)
# static_predicted_all = []
# for key in static_predicted_results:
#     if 'predicted' in key:
#         static_predicted_all.extend(static_predicted_results[key])
# static_predicted_results_pyrightm = list(set(static_predicted_all))
if method == 'autocoderover':
    prediction_path = "/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240621_autocoderover-v20240620/all_preds.jsonl"
elif method == 'aider':
    prediction_path = "/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240523_aider/all_preds.jsonl"
elif method == 'codestory-mixed':
    prediction_path = "/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240702_codestory_aide_mixed/all_preds.jsonl"
elif method == 'Demo':
    prediction_path = "/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240627_abanteai_mentatbot_gpt4o/all_preds.jsonl"
elif method == 'droid':
    prediction_path = "/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240617_factory_code_droid/all_preds.jsonl"
elif method == 'opus_func_margin':
    prediction_path = "~/predictions/opus_func_margin.jsonl"
elif method == 'lingma':
    prediction_path = '/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240622_Lingma_Agent/all_preds.jsonl'
elif method == 'marscode':
    prediction_patch = '/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240723_marscode-agent-dev/all_preds.jsonl'
else:
    raise ValueError("Invalid method")
preds = pd.read_json(prediction_path, lines=True)
instances = preds['instance_id'].to_list()

with open(f'predicted_{method}_results_pylint.json', 'r') as f:
    static_predicted_pylint = json.load(f)
static_predicted_all_pylint = []
for key in static_predicted_pylint:
    if 'predicted' in key:
        static_predicted_all_pylint.extend(static_predicted_pylint[key])
static_predicted_results_pylint = list(set(static_predicted_all_pylint))
static_predicted_pylint_aggressive = []
for instance_id, modified_issues in zip(instances, static_predicted_pylint['original_issues_lsit']):
    if len(modified_issues) > 0:
        static_predicted_pylint_aggressive.append(instance_id)

with open(f'predicted_{method}_results_new.json', 'r') as f:
    static_predicted_pyright = json.load(f)
static_predicted_all_sa = []
for key in static_predicted_pyright:
    if 'predicted' in key:
        static_predicted_all_sa.extend(static_predicted_pyright[key])
static_predicted_results_pyright = list(set(static_predicted_all_sa))
static_predicted_pyright_aggressive = []
for instance_id, modified_issues in zip(instances, static_predicted_pyright['original_issues']):
    if len(modified_issues) > 0:
        static_predicted_pyright_aggressive.append(instance_id)

if method == 'autocoderover':
    method_tmp = 'autocoderover-v20240620-gpt-4o-2024-05-13'
    file_path = f'runtime_detection_predicted_{method_tmp}_results_newpatch_singleshot.jsonl'
else:
    file_path = f'runtime_detection_predicted_{method}_results_newpatch_singleshot.jsonl'
predicted_results = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        predicted_results.append(json.loads(line.strip()))
all_predicted_results = [result for result in predicted_results if result['instance_id'] in all_instances]
# filtered_predicted_results_pyrightm = [result for result in all_predicted_results if result['instance_id'] not in static_predicted_results_pyrightm]
# filtered_predicted_results_pyright = [result for result in all_predicted_results if result['instance_id'] not in static_predicted_results_pyright]
# filtered_predicted_results_pylint = [result for result in all_predicted_results if result['instance_id'] not in static_predicted_results_pylint]

In [39]:
len(predicted_results)

300

In [40]:
def compute_cm(all_instances, eval_results, predicted_all):
    predicted_binary_error = []
    true_binary_error = []
    predicted_list = []
    true_positives = []
    true_negatives = []
    false_positives = []
    false_negatives = []
    for instance in all_instances:
        if instance not in eval_results['passed_instances'] and instance in predicted_all:
            true_positives.append(instance)
        elif instance in eval_results['passed_instances'] and instance in predicted_all:
            false_positives.append(instance)
        elif instance in eval_results['passed_instances'] and instance not in predicted_all:
            true_negatives.append(instance)
        else:
            false_negatives.append(instance)

    return true_positives, false_positives, true_negatives, false_negatives

In [41]:
pyright_true_positives, pyright_false_positives, pyright_true_negatives, pyright_false_negatives = compute_cm(
    all_instances, 
    eval_results, 
    static_predicted_results_pyright)

In [42]:
unsafe_instances_all = []
for pred in all_predicted_results:
    if 'Unsafe' in pred['conclusion']:
        unsafe_instances_all.append(pred['instance_id'])
        print(pred['response'])
safe_instances_all = []
for pred in all_predicted_results:
    if 'Safe' in pred['conclusion']:
        safe_instances_all.append(pred['instance_id'])
llm_true_positives, llm_false_positives, llm_true_negatives, llm_false_negatives = compute_cm(all_instances, eval_results, unsafe_instances_all)

I've analyzed the modification patch and the original implementation. Here's my assessment:

<Runtime Errors>
[AttributeError]: The modification changes `opts = orig_opts` to `opts = cur_model._meta.concrete_model._meta`. This assumes that `cur_model._meta.concrete_model` always exists and has a `_meta` attribute. However, if `cur_model` is already a concrete model, `concrete_model` might return `None`, leading to an AttributeError when trying to access `._meta` on `None`.

[TypeError]: The modification assumes that `cur_model._meta.concrete_model._meta` will always return a valid `Options` object. If for some reason `concrete_model` returns an unexpected type, it could lead to a TypeError when the code tries to use `opts` as an `Options` object in subsequent operations.
</Runtime Errors>

<Conclusion>
Unsafe
</Conclusion>

The modification introduces potential runtime errors that were not present in the original implementation. The original code used `orig_opts`, which was a known val

In [43]:
len(safe_instances_all)

127

In [44]:
# sa missed, llm succeeded
sa_missed_llm_succeeded = [item for item in pyright_false_negatives if item in llm_true_positives]
sa_missed_llm_succeeded.sort()
sa_missed_llm_succeeded

['astropy__astropy-14182',
 'django__django-11283',
 'django__django-11630',
 'django__django-12113',
 'django__django-12284',
 'django__django-12286',
 'django__django-12308',
 'django__django-13220',
 'django__django-13265',
 'django__django-13315',
 'django__django-13660',
 'django__django-14667',
 'django__django-14997',
 'django__django-15213',
 'django__django-15252',
 'django__django-15320',
 'django__django-15400',
 'django__django-15695',
 'django__django-15814',
 'django__django-15819',
 'django__django-15851',
 'django__django-15996',
 'django__django-16229',
 'django__django-16408',
 'django__django-16816',
 'django__django-16820',
 'django__django-16910',
 'matplotlib__matplotlib-22711',
 'matplotlib__matplotlib-23476',
 'matplotlib__matplotlib-24265',
 'matplotlib__matplotlib-24970',
 'matplotlib__matplotlib-25442',
 'mwaskom__seaborn-3407',
 'pallets__flask-4992',
 'pallets__flask-5063',
 'psf__requests-2148',
 'psf__requests-2317',
 'pydata__xarray-3364',
 'pydata__xarr

In [45]:
# sa failed, llm missed
sa_missed_llm_missed = [item for item in pyright_false_negatives if item in llm_false_negatives]
sa_missed_llm_missed.sort()
sa_missed_llm_missed

['django__django-11019',
 'django__django-11905',
 'django__django-12497',
 'django__django-12747',
 'django__django-12908',
 'django__django-13028',
 'django__django-13033',
 'django__django-13448',
 'django__django-13768',
 'django__django-13925',
 'django__django-15061',
 'django__django-15202',
 'django__django-15781',
 'django__django-16400',
 'django__django-17051',
 'matplotlib__matplotlib-22835',
 'matplotlib__matplotlib-23299',
 'matplotlib__matplotlib-25433',
 'pallets__flask-4045',
 'psf__requests-1963',
 'psf__requests-863',
 'pylint-dev__pylint-5859',
 'pylint-dev__pylint-7228',
 'pytest-dev__pytest-5221',
 'pytest-dev__pytest-6116',
 'pytest-dev__pytest-7168',
 'pytest-dev__pytest-7490',
 'scikit-learn__scikit-learn-10508',
 'scikit-learn__scikit-learn-13142',
 'sphinx-doc__sphinx-7975',
 'sphinx-doc__sphinx-8713',
 'sphinx-doc__sphinx-8721',
 'sphinx-doc__sphinx-8801',
 'sympy__sympy-12481',
 'sympy__sympy-13031',
 'sympy__sympy-13043',
 'sympy__sympy-13437',
 'sympy__sy

In [46]:
# sa good, llm false
sa_good_llm_false = [item for item in pyright_true_negatives if item in llm_false_positives]
sa_good_llm_false.sort()
sa_good_llm_false

['django__django-11001',
 'django__django-11179',
 'django__django-11422',
 'django__django-12708',
 'django__django-13401',
 'django__django-13447',
 'django__django-13551',
 'django__django-13590',
 'django__django-14016',
 'django__django-14238',
 'django__django-14580',
 'django__django-14608',
 'django__django-14999',
 'django__django-15388',
 'django__django-16139',
 'django__django-17087',
 'matplotlib__matplotlib-24149',
 'matplotlib__matplotlib-24334',
 'matplotlib__matplotlib-26011',
 'pytest-dev__pytest-7220',
 'pytest-dev__pytest-7373',
 'pytest-dev__pytest-8365',
 'scikit-learn__scikit-learn-10297',
 'scikit-learn__scikit-learn-14087',
 'sphinx-doc__sphinx-10325',
 'sympy__sympy-13471',
 'sympy__sympy-15011',
 'sympy__sympy-18057']

In [47]:
llm_true_negatives

['django__django-11999',
 'scikit-learn__scikit-learn-15512',
 'matplotlib__matplotlib-23314',
 'pytest-dev__pytest-5227',
 'scikit-learn__scikit-learn-12471',
 'pytest-dev__pytest-5692',
 'sphinx-doc__sphinx-8595',
 'django__django-12453',
 'pytest-dev__pytest-11143',
 'scikit-learn__scikit-learn-13496',
 'django__django-12983',
 'astropy__astropy-14365',
 'django__django-16046',
 'django__django-13710',
 'pylint-dev__pylint-7993',
 'django__django-14672',
 'sympy__sympy-17655',
 'django__django-16595',
 'django__django-13658',
 'scikit-learn__scikit-learn-14894',
 'matplotlib__matplotlib-23964',
 'scikit-learn__scikit-learn-13241',
 'django__django-11039',
 'django__django-11133',
 'django__django-14382',
 'django__django-16527',
 'matplotlib__matplotlib-23913',
 'django__django-12700',
 'sympy__sympy-24213',
 'django__django-14411',
 'sympy__sympy-20154',
 'pytest-dev__pytest-7432',
 'astropy__astropy-14995',
 'scikit-learn__scikit-learn-14983',
 'pylint-dev__pylint-6506',
 'scikit-

In [48]:
instance_id = 'sympy__sympy-13471'
pred = [item for item in predicted_results if item['instance_id'] == instance_id][0]
patch = preds[preds['instance_id']==instance_id]['model_patch'].item()
# repo = pred['repo']
# base_commit = pred['base_commit']
# dir = '/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240702_codestory_aide_mixed/logs'
dir = '/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240627_abanteai_mentatbot_gpt4o/logs'
# dir = '/home/ubuntu/mnt/agent/experiments/evaluation/lite/20240622_Lingma_Agent/logs'
template = "{instance_id}.20240627_abanteai_mentatbot_gpt4o.eval.log"
# template = '{instance_id}.20240622_Lingma_Agent.eval.log'
# template = '{instance_id}.20240702_codestory_aide_mixed.eval.log'
file_name = template.format(
    instance_id = instance_id
)
with open(f'{dir}/{file_name}', 'r') as f:
    log_content = f.read()

In [49]:
print(pred['conclusion'])

Unsafe


In [50]:
print(pred['response'])

I've analyzed the modification patch and the original implementation. Here's my assessment:

<Runtime Errors>
[TypeError]: The modification assumes that num[1] is always a string, but it might not be. If num[1] is already an integer (long), calling .rstrip('L') on it will raise a TypeError.
</Runtime Errors>

<Conclusion>Unsafe</Conclusion>

The modification patch introduces a potential runtime error. It assumes that num[1] is always a string and tries to call the .rstrip('L') method on it. However, in the original implementation, num[1] could already be an integer (long) if the input comes from a non-pickled source. In such cases, calling .rstrip('L') on an integer would raise a TypeError.

To make this modification safe, it should first check if num[1] is a string before attempting to strip the 'L' suffix. For example:

```python
if isinstance(num[1], string_types):
    num[1] = num[1].rstrip('L')  # Strip 'L' suffix if present
num[1] = long(num[1], 16)
```

This change would make th

In [51]:
print(patch)

diff --git a/sympy/core/numbers.py b/sympy/core/numbers.py
index 7bc96e68eb..4c8301c6b2 100644
--- a/sympy/core/numbers.py
+++ b/sympy/core/numbers.py
@@ -1042,6 +1042,7 @@ def __new__(cls, num, dps=None, prec=None, precision=None):
                 # it's a hexadecimal (coming from a pickled object)
                 # assume that it is in standard form
                 num = list(num)
+                num[1] = num[1].rstrip('L')  # Strip 'L' suffix if present
                 num[1] = long(num[1], 16)
                 _mpf_ = tuple(num)
             else:



In [52]:
print(log_content)

[sympy__sympy__1.1] [sympy__sympy-13471] Task Metadata:
	- Instance ID: sympy__sympy-13471
	- Testbed: sympy__sympy__1.1
	- 
	- Evaluation Model: Demo 
[sympy__sympy__1.1] [sympy__sympy-13471] Command: git status --porcelain 
[sympy__sympy__1.1] [sympy__sympy-13471] Subprocess args: {"cwd": "/home/swe-bench/sympy__sympy", "check": true, "shell": false, "capture_output": false, "text": true, "stdout": -1, "stderr": -2} 
[sympy__sympy__1.1] [sympy__sympy-13471] Std. Output:
 
[sympy__sympy__1.1] [sympy__sympy-13471] Return Code: 0 
[sympy__sympy__1.1] [sympy__sympy-13471] Command: git -c advice.detachedHead=false checkout 3546ac7ed78e1780c1a76929864bb33330055740 
[sympy__sympy__1.1] [sympy__sympy-13471] Subprocess args: {"cwd": "/home/swe-bench/sympy__sympy", "check": true, "shell": false, "capture_output": false, "text": true, "stdout": -1, "stderr": -2} 
[sympy__sympy__1.1] [sympy__sympy-13471] Std. Output:
Previous HEAD position was ec9e3c0436 Merge pull request #14817 from jashan498/